# Trabajo Final

###  CLASIFICACION CON KNN

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

### Integrantes:

Nombre: David de Jesus Yepes Isaza

Cédula: 1020455542

Nombre: Rodrigo Antonio Porras Martinez

Cédula: 1039459720

### Clasificación en el dataset "Heart Disease Data Set" con KNN

La base de datos con las que se trabaja en el presente artículo fue extraída de  Cleveland Clinic Foundation y fue creada por V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.

El set de datos de de Cleveland consta de 303 instancias, la variable de salida o etiqueta son 0 para la ausencia y los valores 1, 2, 3 y 4 para la presencia de enfermedades. 
 
El 54% de las instancias tienen ausencia de enfermedades y 46% tienen presencia de enfermedades del corazón. Para simplificar el problema se analiza como un problema biclase donde  “0” son personas sanas y “1” que corresponde a (1,2,3,4) son personas enfermas.

El objetivo es según las características de cada paciente identificar cuál de ellos presenta presencia de enfermedades y cuáles no.

La base de datos contiene 303 muestras con las 14 características siguientes:
1. N° 3 (edad)   
2. N° 4 (sexo) 
3. N° 9 (cp) tipo de dolor en el pecho
        Valor 1: angina típica
        Valor 2: angina atípica
        Valor 3: dolor no anginal
        Valor 4: asintomático  
4. N° 10 (trestbps) presión arterial en reposo
5. N° 12 (chol) colesterol sérico en mg / dl (miligramo por decilitro)
6. N° 16 (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
7. N° 19 (restecg)  resultados electrocardiográficos en reposo:
        Valor 0: normal
        Valor 1: tiene anormalidad de onda ST-T (inversiones de onda T y / o ST  elevación o depresión de > 0.05 mV)
        Valor 2: muestra hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes  
8. N° 32 (thalach)  frecuencia cardíaca máxima lograda
9. N° 38 (exang) angina inducida por el ejercicio (1 = sí; 0 = no)
10. N° 40 (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo
11. N° 41 (pendiente) la pendiente del segmento ST de ejercicio máximo
        Valor 1: pendiente ascendente
        Valor 2: plano
        Valor 3: downsloping
12. N° 44 (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. N° 51 (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible
14. N° 58 (num) (el atributo predicho) diagnóstico de enfermedad cardíaca (estado angiográfico de la enfermedad)
        - Valor 0: <50% de diámetro de estrechamiento
        - Valor 1:> 50% de diámetro de estrechamiento

### Importacion de Librerias, metodo KNN

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from scipy import stats
from scipy.spatial import distance
from scipy.stats import mode
import operator
import time
from sklearn.metrics import confusion_matrix

#Método para calcular la distancia entre dos vectores
def dist_euclidiana(x1, x2):
    
    dist = distance.euclidean(x1, x2)
    
    return dist

#Para calcular el error en los problemas de clasificación
def error(Y_lest, Y):
    error = 0
    for ye, y in zip(Y_lest, Y):
        if ye != y:
            error += 1
    
    error = error/np.size(Y)
    
    return error

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

def KNN(X_train, Y_train, X_val, Y_val, knn, tipo):
    
    k = knn    #Parámetro k que equivale al número de vecinos a tener en cuenta para resolver el problema de
               #predicción de la variable de salida
    
    Y_est = []
    
    for x1 in X_val:
        distances = dict()
        for x2, y2 in zip(X_train, Y_train):
            distances.update({dist_euclidiana(x1,x2): y2})
    
        #k vecinos más cercanos cercanos
        vecinos = sorted(distances.items(), key=operator.itemgetter(0))
        #print vecinos
        
        #Complete aquí el código para tomar solo la variables de salida de los k vecinos más cercanos
        y_vecinos = []
        for v in vecinos[:knn]:
            y_vecinos.append(v[1])
        
        #Se calcula la variable de salida
        #Regression
        if tipo == 'regression':
            #Complete el código aquí para calcular la variable de salida en el problema de regresión
            #y_est = 
            Y_est.append(y_est)
        #Classification
        elif tipo == 'classification':
            #Complete el código aquí para calcular la variable de salida en el problema de clasificación
            mode = stats.mode(y_vecinos, axis=None)
            #print(mode)
            #print('mode[0]:'+mode[0])
            y_est = mode[0]
            Y_est.append(y_est[0])
            #print(Y_est)
    
    '''if tipo == 'regression':
        print ("Modelo KNN para regresión\n\n"+"El Error cuadrático medio del modelo con " + str(k) + " vecinos es = " + str(ECM(Y_est, Y_val)))
    elif tipo == 'classification':
        
        #print ("El Error de clasificación del modelo con " + str(k) + " vecinos es = " + str(error(Y_est,Y_val))+"\n")
        '''
    return Y_est
        


### Eliminacion de valores Perdidos (Missing Values)

En la base de datos algunos valores están perdidos, se pueden identificar por el valor “-9.0”. por lo que se deben limpiar.

In [3]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize

clevelanddb = np.loadtxt('cleveland.txt',delimiter=',')  # Assuming comma-delimiter
print ("Dim de la base de datos original: " + str(np.shape(clevelanddb)))
DataBase = clevelanddb
#La base de datos cleveland.txt tiene algunos valores perdidos, estos se identifican con -9.0
#Los valores de salida mayores a 0 se considerarán 1 de esta forma la clasificacion será biclase 
j = 0
num_reset_values = 0

for i in range(0,np.size(clevelanddb,0)):
    if -9.0 == clevelanddb[i,13]:
        j+=1
        DataBase = np.delete(DataBase,i,0)
    elif clevelanddb[i,13] > 0:
        clevelanddb[i,13] = 1
        num_reset_values += 1

print ("\nHay " + str(j) + " valores perdidos en la variable de salida.")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido " + str(np.shape(DataBase)))

print ("Se han reseteado a 1 "+str(num_reset_values)+" valores ya que eran mayores a 0")

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,14):
        if -9.0 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -9.0 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")


Dim de la base de datos original: (303, 14)

Hay 0 valores perdidos en la variable de salida.

Dim de la base de datos sin las muestras con variable de salida perdido (303, 14)
Se han reseteado a 1 139 valores ya que eran mayores a 0

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase


### Aplicando KNN con Cross validation, 10 folds y 1,3,5 y 7 vecinos para cada fold.


In [4]:

X = DataBase[:,0:13]
Y = DataBase[:,13]


#Normalizamos los datos
X = normalize(X, axis=0, norm='l1')



#Implemetamos la metodología de validación cross validation con 10 folds

#Para calcular el costo computacional
tiempo_i = time.time()

Errores = np.ones(10)
Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)

j = 0
fold_num = 0
kf = KFold(n_splits=10)
tipo="classification"
num_vecinos = 7
print ("Ejecucion con 10 folds y "+str(num_vecinos)+" Vecinos")

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    fold_num += 1
    #print ("Fold numero: " + str(fold_num))
    
    ypred =  KNN(X_train,y_train,X_test,y_test,num_vecinos,tipo)

    tn, fp, fn, tp = confusion_matrix(y_test,ypred).ravel()

    Errores[j] = classification_error(ypred, y_test)
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
        
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))       
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia : "+str(np.mean(Eficiencia)) + "\n ")
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")



Ejecucion con 10 folds y 7 Vecinos

Error de validación: 0.22774193548387095 +/- 0.08545891297256016

Sensibilidad: 0.7394623758594348

Especificidad: 0.8007257963914309

Precision: 0.7723126873126873

Eficiencia : 0.772258064516129
 


Tiempo total de ejecución: 3.51516795158 segundos.


  
<table>
  <tr>
    <th>FOLDS</th>
    <th>NUMERO DE VECINOS</th>
    <th>ERROR DE VALIDACION</th>
    <th>SENSIBILIDAD</th>
    <th>ESPECIFICIDAD</th>
    <th>PRECISION</th>
    <th>EFICIENCIA</th>
    <th>TIEMPO TOTAL DE EJCUCION</th>
    
  </tr>
  <tr>
    <td>10</td>
    <td>1</td>
    <td>0.2248387096774193 +/- 0.09293265414036607</td>
    <td>0.7484842118665649</td>
    <td>0.7911295489855862</td>
    <td>0.7713578088578089</td>
    <td>0.7751612903225806</td>
    <td>3.73157405853 segundos.</td>
  </tr>
  <tr>
    <td>10</td>
    <td>3</td>
    <td>0.20473118279569893 +/- 0.07800880739813912</td>
    <td>0.7598908199643494</td>
    <td>0.8261914826659409</td>
    <td>0.7958275058275058</td>
    <td>0.7952688172043011</td>
    <td>4.34851908684 segundos.</td>
  </tr>
  <tr>
    <td>10</td>
    <td>5</td>
    <td>0.21451612903225806 +/- 0.07607441391669807</td>
    <td>0.7667580213903744</td>
    <td>0.794902683745563</td>
    <td>0.7647504456327987</td>
    <td>0.785483870967742</td>
    <td>4.43060612679 segundos.</td>
  </tr>
  <tr>
    <td>10</td>
    <td>7</td>
    <td>0.22774193548387095 +/- 0.08545891297256016</td>
    <td>0.7394623758594348</td>
    <td>0.8007257963914309</td>
    <td>0.7723126873126873</td>
    <td>0.772258064516129</td>
    <td>3.74804997444 segundos.</td>
  </tr>
</table>

## Selección de Caracteristicas - Wrappers

Para selecionar las caracteristicas usaremos la funcion Wrappers con KNN. los parámetros son el número de características a seleccionar  (3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS)


In [30]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier

#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False

#SFS True False, SBS False False, SFFS True True, SBFS False True
m_forward = False
m_floating = True
m_features = 7

scores =['accuracy', 'precision', 'recall']

def select_features(modelo, n_features, fwd, fltg,score):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring = score,
           cv=0)
    
    return sfs


#Para calcular el costo computacional
tiempo_i = time.time()

#Implemetamos la metodología de validación cross validation con 10 folds

Medidas = np.ones(10)
MedidasPromedio = np.ones(3)
k = 0

j = 0

kf = KFold(n_splits=10)

Errores = np.ones(10) 


#se utiliza KNN de SKlearn
neigh = KNeighborsClassifier(n_neighbors=3)
for scr in scores:
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        #SFS True False, SBS False False, SFFS True True, SBFS False True
        sf = select_features(neigh,m_features,m_forward,m_floating,scr)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf = sf.fit(X_train,y_train)

        Medidas[j] = sf.k_score_
        j+=1
    MedidasPromedio[k]=np.mean(Medidas)
    k+=1
    j=0

print("SFS = True False, SBS = False False, SFFS = True True, SBFS = False True")
print (str(m_forward)+"-"+str(m_floating) )
print("# Caracteristicas a elegir: "+str(m_features))
print("\nError en validación : " + str((1-MedidasPromedio[0])*100) + "%" )
print("\nEficiencia : " + str(MedidasPromedio[0]*100) + "%" )
print("\nPresicion : " + str(MedidasPromedio[1]*100) + "%" )
print("\nSensibilidad : " + str(MedidasPromedio[2]*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

print("Las caracteristicas seleccionadas son: ")

dictChar = {0 : "1. edad", 
            1 : '2. sexo',
            2 : '3. (cp) tipo de dolor en el pecho',
            3 : '4. (trestbps) presión arterial en reposo',
            4 : '5. (chol) colesterol sérico en mg / dl (miligramo por decilitro)',
            5 : '6. (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)',
            6 : '7. (restecg) resultados electrocardiográficos en reposo:',
            7 : '8. (thalach) frecuencia cardíaca máxima lograda',
            8 : '9. (exang) angina inducida por el ejercicio (1 = sí; 0 = no)',
            9 : '10. (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo',
            10 :'11. (pendiente) la pendiente del segmento ST de ejercicio máximo',
            11 :'12. (ca) número de vasos principales (0-3) coloreados por fluoroscopia',
            12 :'13. (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible'}

for x in range(0,len(sf.k_feature_idx_)):
    print (dictChar[sf.k_feature_idx_[x]])

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.0s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s fini

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.0s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s fini

Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s fini

Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:

SFS = True False, SBS = False False, SFFS = True True, SBFS = False True
False-True
# Caracteristicas a elegir: 7

Error en validación : 10.524402068519711%

Eficiencia : 89.4755979314803%

Presicion : 90.19318190429324%

Sensibilidad : 91.0520019572816%


Tiempo total de ejecución: 12.0986878872 segundos.
Las caracteristicas seleccionadas son: 
1. edad
2. sexo
6. (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
9. (exang) angina inducida por el ejercicio (1 = sí; 0 = no)
11. (pendiente) la pendiente del segmento ST de ejercicio máximo
12. (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible


  
<table>
  <tr>
    <th># FEATURES</th>
    <th>TECNICA</th>
    <th>ERROR</th>
    <th>EFICIENCIA </th>
    <td>PRECISION</td>
    <td>SENSIBILIDAD</td>
    <td>TIEMPO DE EJECUCION(S)</td>
  </tr>
  <tr>
    <td>3</td>
    <td>KNN + SFS</td>
    <td>14.117916397328155%</td>
    <td>85.88208360267184%</td>
    <td>86.61733143078465%</td>
    <td>85.34833650948222%</td>
    <td>4.35003113747</td>
 </tr>
 <tr>
    <td>3</td>
    <td>KNN + SBS</td>
    <td>14.814695108812758%</td>
    <td>85.18530489118724%</td>
    <td>85.8216783931255%</td>
    <td>83.67502090292915%</td>
    <td>12.9563469887</td>
 </tr>
  <tr>
    <td>3</td>
    <td>KNN + SFFS</td>
    <td>14.117916397328155%</td>
    <td>85.88208360267184%</td>
    <td>86.61733143078465%</td>
    <td>85.34833650948222%</td>
      <td>4.65810108185</td>
 </tr>
  <tr>
    <td>3</td>
    <td>KNN + SBFS</td>
    <td>13.898136177547936%</td>
    <td>86.10186382245206%</td>
    <td>85.37280016663222%</td>
    <td>84.296845089921%</td>
    <td>26.3281989098</td>
 </tr>
 <tr>
    <td>5</td>
    <td>KNN + SFS</td>
    <td>11.916882137470374%</td>
    <td>88.08311786252962%</td>
    <td>89.58087593238581%</td>
    <td>87.68998013697747%</td>
     <td>6.73341417313</td>
 </tr>
 <tr>
    <td>5</td>
    <td>KNN + SBS</td>
    <td>11.514625080801544%</td>
    <td>88.48537491919845%</td>
    <td>88.20208322579755%</td>
    <td>88.80405758932986%</td>
     <td>11.0602440834</td>
 </tr>
  <tr>
    <td>5</td>
    <td>KNN + SFFS</td>
    <td>11.770361990950228%</td>
    <td>88.22963800904976%</td>
    <td>89.85623231001492%</td>
    <td>88.4112704595581%</td>
      <td>8.3667550087</td>
 </tr>
  <tr>
    <td>5</td>
    <td>KNN + SBFS</td>
   <td>11.33120555914674%</td>
    <td>88.66879444085326%</td>
    <td>89.02753492960832%</td>
    <td>88.80405758932986%</td>
      <td>18.4757401943</td>
 </tr>
 <tr>
    <td>7</td>
    <td>KNN + SFS</td>
    <td>10.634830855419096%</td>
    <td>89.3651691445809%</td>
    <td>89.25613699379169%</td>
    <td>88.9773742734089%</td>
     <td>8.78285098076</td>
 </tr>
 <tr>
    <td>7</td>
    <td>KNN + SBS</td>
    <td>10.451411333764272%</td>
    <td>89.54858866623573%</td>
    <td>90.25565499444328%</td>
    <td>91.0520019572816%</td>
     <td>9.34508609772</td>
 </tr>
  <tr>
    <td>7</td>
    <td>KNN + SFFS</td>
    <td>10.524940745528987%</td>
    <td>89.475059254471%</td>
    <td>89.93165259261191%</td>
    <td>90.33214688049512%</td>
      <td>14.3637700081</td>
 </tr>
  <tr>
    <td>7</td>
    <td>KNN + SBFS</td>
    <td>10.524402068519711%</td>
    <td>89.4755979314803%</td>
    <td>90.19318190429324%</td>
    <td>91.0520019572816%</td>
      <td>12.0986878872</td>
 </tr>
</table>

In [40]:
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA

#Feature Extraction Function
#Recibe 2 parámetros: 1. el tipo de método de extracción (pca o lda como string), 2. el número componentes (para pca)
#o el número de discriminantes (para lda)

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

#Para calcular el costo computacional
tiempo_i = time.time()

X = DataBase[:,0:13]
#Estandarizamos los datos
X = standardize(X)

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)

j = 0
kf = KFold(n_splits=10)

#se utiliza KNN de SKlearn
neigh = KNeighborsClassifier(n_neighbors=3)

for train_index, test_index in kf.split(X):
    
    #Aquí se aplica la extracción de características por PCA
    ex = extract_features('pca',12) 

    #Fit de PCA
    ex = ex.fit(X)
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    X_ex = ex.transform(X)
    
    #Se aplica CV-10
    
    X_train, X_test = X_ex[train_index], X_ex[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    
    # Entrenamiento el modelo.
    model = neigh.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)

    tn, fp, fn, tp = confusion_matrix(y_test,ypred).ravel()

    Errores[j] = classification_error(ypred, y_test)
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
        

print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))       
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia : "+str(np.mean(Eficiencia)) + "\n ")
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")




Error de validación: 0.16838709677419356 +/- 0.060483655913596175

Sensibilidad: 0.8361946142093201

Especificidad: 0.8237133132605269

Precision: 0.810101760984114

Eficiencia : 0.8316129032258065
 


Tiempo total de ejecución: 0.0410730838776 segundos.



<table>
  <tr>
    <th># COMPONENTES</th>
    <th>TECNICA</th>
    <th>ERROR</th>
     <td>SENSIBILIDAD</td>
     <td>ESPECIFICIDAD</td>
      <td>PRECISION</td>
    <th>EFICIENCIA </th>
    <td>TIEMPO DE EJECUCION(S)</td>
  </tr>
  <tr>
    <td>2</td>
    <td>KNN + PCA</td>
    <td>0.2906451612903226 +/- 0.04193562172571772</td>
    <td>0.6663340336134455</td>
    <td>0.7576234704408079</td>
    <td>0.7033863685334273</td>
    <td>0.7093548387096774</td>
     <td>0.0415890216827</td>
    
 </tr>
 <tr>
    <td>6</td>
    <td>KNN + PCA</td>
    <td>0.23741935483870966 +/- 0.06841076211887147</td>
    <td>0.7560160427807486</td>
    <td>0.7759797400741674</td>
    <td>0.7396463585434174</td>
    <td>0.7625806451612903</td>
     <td>0.0387468338013</td>
 </tr>
    <tr>
    <td>10</td>
    <td>KNN + PCA</td>
    <td>0.2174193548387097 +/- 0.07089080221598074</td>
    <td>0.7734832569391393</td>
    <td>0.7904844690912802</td>
    <td>0.7641666666666667</td>
    <td>0.7825806451612903</td>
     <td>0.0400590896606</td>
 </tr>
    <tr>
    <td>12</td>
    <td>KNN + PCA</td>
    <td>0.19139784946236557 +/- 0.06937775732016574</td>
    <td>0.7915947288006112</td>
    <td>0.8127377975481691</td>
    <td>0.803721572545102</td>
    <td>0.8086021505376344</td>
     <td>0.0326619148254</td>
 </tr>
</table>